In [29]:
import os
import random
import shutil
import numpy as np
from tqdm import tqdm
from collections import Counter
from pathlib import Path

In [69]:
EXERCISE = 'half-burpee'
TYPE = 'images'
LEN = 8
SAM_NUM = 252

In [70]:
if TYPE != 'images':
    neg_dir = f'/home/u6188041/jom/Dataset/EzFit_dataset/{TYPE}/len_{LEN}'
    accepted_exts = ('.npy')
else:
    neg_dir = f'/home/u6188041/jom/Dataset/EzFit_dataset/{TYPE}'
    accepted_exts = ('.jpg', '.jpeg','.png')
all_negs = []
classes = []
for path, subdir, files in os.walk(neg_dir):
    for file in files:
        full_path = os.path.join(path, file)
        if file.endswith(accepted_exts) and EXERCISE not in full_path and 'other' not in full_path and 'idle' not in full_path:
            split = full_path.split(os.path.sep)
            if TYPE != 'images':
                class_name = split[-4] + '_' + split[-3]
            else:
                class_name = split[-3] + '_' + split[-2]
            
            all_negs.append(full_path)
            classes.append(class_name)

In [71]:
all_negs

['/home/u6188041/jom/Dataset/EzFit_dataset/images/leg-raise/leg-raise-up/64_2_1.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/leg-raise/leg-raise-up/16_4_1.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/leg-raise/leg-raise-up/67_1_2.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/leg-raise/leg-raise-up/13_2_1.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/leg-raise/leg-raise-up/86_2_1.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/leg-raise/leg-raise-up/68_1_2.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/leg-raise/leg-raise-up/20_4_1.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/leg-raise/leg-raise-up/101_1_2.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/leg-raise/leg-raise-up/100_2_1.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/leg-raise/leg-raise-up/97_2_1.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/leg-raise/leg-raise-up/36_2_1.jpg',
 '/home/u6188041/jom/Dataset/EzFit_dataset/images/le

In [72]:
len(all_negs)

2432

In [73]:
total = len(classes)
sam_weights_mapping = {k:total/v for k,v in dict(Counter(classes)).items()}
# sam_weights_mapping
sam_weights = [sam_weights_mapping[cls] for cls in classes]

In [74]:
class_mapping = dict(zip(all_negs, classes))
weight_mapping = dict(zip(all_negs, sam_weights))

In [75]:
picked = random.choices(all_negs, weights=sam_weights, k=SAM_NUM)

In [76]:
picked_classes = [class_mapping[path] for path in picked]
picked_weights = [weight_mapping[path] for path in picked] # acutually... useless

In [77]:
# check distribution of sample
Counter(picked_classes)

Counter({'squat_squat-up': 42,
         'pushup_pushup-down': 29,
         'pushup_pushup-up': 25,
         'jumping-jack_jumping-jack-up': 22,
         'leg-raise_leg-raise-up': 41,
         'jumping-jack_jumping-jack-down': 28,
         'squat_squat-down': 26,
         'leg-raise_leg-raise-down': 39})

# Save to .npy files

In [78]:
if TYPE != 'images':
    save_dir = Path(f'/home/u6188041/jom/Dataset/EzFit_dataset/{TYPE}/len_{LEN}/{EXERCISE}/others')
else:
    save_dir = Path(f'/home/u6188041/jom/Dataset/EzFit_dataset/{TYPE}/{EXERCISE}/others')
for idx in tqdm(range(len(picked))):
    if TYPE != 'images':
        base_dir = save_dir / str(idx)
    else:
        base_dir = save_dir
    if not base_dir.exists():
        base_dir.mkdir(parents=True)
    selected = Path(picked[idx])
    selected_ext = selected.suffix # this will return extension (e.g. .jpg, .npy)
    shutil.copy2(selected, base_dir / f'others_{idx}{selected_ext}')

100%|███████████████████████████████████████████████████████████████████████████████| 252/252 [00:00<00:00, 6262.79it/s]
